In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
(img_train,label_train),(img_test,label_test)=keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [8]:
#zdefiniowanie modelu z hiperparametrami
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
  model.add(keras.layers.Dense(units=hp_units,activation='relu'))

  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate',values=[1.0,1e-2,1e-3,1e-4,1e-5])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [9]:
#utowrzenie wystąpienia tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [11]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

print(f"""
        Wyszukiwanie hiperparametrów zakończone.
        Najlepsze hiperparametry:
        Liczba neuronów: {best_hps.get('units')}
        Szybkość uczenia: {best_hps.get('learning_rate')}
""")

Trial 30 Complete [00h 02m 49s]
val_accuracy: 0.1029166653752327

Best val_accuracy So Far: 0.8879166841506958
Total elapsed time: 00h 23m 24s

        Wyszukiwanie hiperparametrów zakończone.
        Najlepsze hiperparametry:
        Liczba neuronów: 128
        Szybkość uczenia: 0.001



In [12]:
model = tuner.hypermodel.build(best_hps)
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_perepch = history.history['val_accuracy']
best_epoch = val_acc_perepch.index(max(val_acc_perepch))+1
print('Najlepsza liczba epok:',best_epoch)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7710 - loss: 0.6608 - val_accuracy: 0.8572 - val_loss: 0.4023
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8552 - loss: 0.4048 - val_accuracy: 0.8570 - val_loss: 0.3905
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8698 - loss: 0.3514 - val_accuracy: 0.8570 - val_loss: 0.3878
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8789 - loss: 0.3248 - val_accuracy: 0.8665 - val_loss: 0.3608
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8877 - loss: 0.3045 - val_accuracy: 0.8812 - val_loss: 0.3331
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8917 - loss: 0.2918 - val_accuracy: 0.8843 - val_loss: 0.3211
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9000 - loss: 0.2738 - val_accuracy: 0.8806 - val_loss: 0.3222
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9036 - loss: 0.2623 